In [117]:
import numpy as np

Integrantes: Jeremias Laria(1329/21) y Malena Alamo(1620/21)

### Ejercicio 2. 
Implementar un programa que dada una matriz $A ∈ R^{n×n}$ devuelva las dos matrices L y U de la descomposicion A = LU. No es necesario implementar pivoteo. En caso de que aparezca un 0 en la diagonal en algun paso del algoritmo, imprimir un mensaje de error y devolver las matrices In y A.

In [118]:
def descomposicionLU(A):
    
    fils=np.shape(A)[0]
    cols=np.shape(A)[1]
    
    #Inicializo L y U, para luego reescribirlas
    L=np.eye(cols)  
    U=np.copy(A)
  
    for j in range(cols):
    
        if(A[j,j]!=0):
            for i in range(j+1,fils):
                L[i,j]=(U[i,j]/U[j,j])
                U[i,:]=U[i,:]+(-U[i,j]/U[j,j])*U[j,:]    
    
        else:
            print('Error') 
            return [np.eye(fils), A] 
            break
  
    return [L, U]

Tomamos $A$ y $B \in R^{4x4} $, $A$ admite descomposicion LU, y $B$ no la admite

In [119]:
A=np.array([[2,4,0,0],[-3,4,5,0],[4,18,6,8],[1,2,0,7]])
descomposicionLU(A)

[array([[ 1. ,  0. ,  0. ,  0. ],
        [-1.5,  1. ,  0. ,  0. ],
        [ 2. ,  1. ,  1. ,  0. ],
        [ 0.5,  0. ,  0. ,  1. ]]),
 array([[ 2,  4,  0,  0],
        [ 0, 10,  5,  0],
        [ 0,  0,  1,  8],
        [ 0,  0,  0,  7]])]

In [120]:
B=np.array([[1,-1,0,1],[-1,0,-2,0],[0,1,1,-1],[3,1,2,-1]])
descomposicionLU(B)

Error


[array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 array([[ 1, -1,  0,  1],
        [-1,  0, -2,  0],
        [ 0,  1,  1, -1],
        [ 3,  1,  2, -1]])]

### Ejercicio 3.
Implementar las funciones del Ejercicio 1 de la Practica 3 para resolver sistemas de
ecuaciones $Ly = b$ y $Ux = y$ suponiendo que las matrices $L$ y $U$ son triangulares inferior y superior
respectivamente y no tienen 0’s en la diagonal.

In [121]:
#Funcion auxiliar para sistemaUsandoLU
def sumaCoeficientesU(U,x,i):
    n=len(U)
    suma=0
    for j in range (i+1,n):
        suma=suma + (U[i,j]*x[j])
    return suma

In [122]:
#Funcion auxiliar para sistemaUsandoLU
def sistemaU (U,b):
    longitud=len(U)
    x=np.zeros(longitud)
    x[longitud-1]=((b[longitud-1])/U[longitud-1,longitud-1])
    for i in range (longitud - 2,-1,-1):
        x[i]=(b[i]-sumaCoeficientesU(U,x,i))/U[i,i]
    return x

In [123]:
#Funcion auxiliar para sistemaUsandoLU
def sumaCoeficientesL(L,y,i):
    suma=0
    for j in range (0,i):
        suma=suma + L[i,j]*y[j]
    return suma

In [124]:
#Funcion auxiliar para sistemaUsandoLU
def sistemaL(L,x):
    longitud=len(L)
    y=np.zeros(longitud)
    y[0]=x[0]/L[0,0]
    for i in range (1,longitud):
        y[i]=(x[i]-sumaCoeficientesL(L,y,i))/L[i,i]
    return y

In [125]:
def sistemaUsandoLU(A,b):
    L, U = descomposicionLU(A)
    y=sistemaL(L,b)
    x=sistemaU(U,y)
    return x

Tomamos $A\in R^{4x4}$ y $b\in R^4$, verificamos que funciona haciendo $Ax=b$

In [126]:
A=np.array([[2,4,0,0],[-3,4,5,0],[4,18,6,8],[1,2,0,7]])
b=np.array([1,2,3,4])
x=sistemaUsandoLU(A,b)

print(np.all(np.isclose(A@x,b)))



True


### Ejercicio 4.
Probar las funciones implementadas tomando una matriz $A ∈ R^{10×10}$ de numeros aleatorios y un vector $b ∈ R^{10}$ de numeros aleatorios. Calcular la norma-1 del error $e = Ax̃ − b$ para x̃ la solucion calculada.

In [127]:
def error(A,b):
    
    x=sistemaUsandoLU(A,b)
    
    return ["La norma 1 del error es: ", np.linalg.norm(A@x-b,ord=1)]

In [128]:
A=np.random.rand(10,10)
b=np.random.rand(10)

error(A,b)

['La norma 1 del error es: ', 4.4603210014315664e-14]

### Ejercicio 5.
Implementar un programa que reciba una matriz $A$ definida positiva (simetrica) y devuelva las matrices $L$ y $D$ de la descomposicion $A=LDL^T$.

In [129]:
def es_definida_pos(A):
    
    for k in range (len(A[0])) :
        if (np.linalg.det(A[:k,:k])<=0): #A es definida positiva si todos sus menores son mayores a 0
            return False
        
    return True

In [130]:
#Funcion auxiliar para descomposicionLDLt
def sumatoriaD(D,A,L,j):
    suma=0
    for k in range (j):
        suma=suma + ((L[j,k])**2)*D[k,k]
        
    return suma

In [131]:
#Funcion auxiliar para descomposicionLDLt
def sumatoriaL(D,A,L,i,j):
    suma=0
    for k in range (j):
        suma=suma + L[i,k]*L[j,k]*D[k,k]
    return suma

In [132]:
def descomposicionLDLt(A):
    n=np.shape(A)[0]
    if (es_definida_pos(A)):
        if((A==A.T).all()):
            D=np.zeros([n,n])
            L=np.eye(n)
            for j in range (n):
                for i in range (j, n):
                    D[j,j]=A[j,j]-sumatoriaD(D,A,L,j)
                    L[i,j]=(1/D[j,j])*(A[i,j]-sumatoriaL(D,A,L,i,j))
            return [L,D]
        else:
            print("La matriz no es simetrica") #A es simetrica si A=A^t
        
    else: 
        print("La matriz debe ser definida positiva")

Tomamos $A ∈ R^{3x3}$ simetrica y definida positiva, $B ∈ R^{3x3}$ definida positiva y no simetrica, y $C ∈ R^{3x3}$ no definida positiva, para $A$ verificamos que $A=LDL^T$

In [133]:
A=np.array([[9,3,1],[3,5,0],[1,0,3]])
descomposicionLDLt(A)[0]@descomposicionLDLt(A)[1]@descomposicionLDLt(A)[0].T

array([[9., 3., 1.],
       [3., 5., 0.],
       [1., 0., 3.]])

In [134]:
B=np.array([[4,1,1],[3,5,0],[1,0,3]])
descomposicionLDLt(B)

La matriz no es simetrica


In [135]:
B=np.array([[1,3,1],[3,5,0],[1,0,3]])
descomposicionLDLt(B)

La matriz debe ser definida positiva


### Ejercicio 6.
Implementar una funcion que dada una matriz diagonal $D ∈ R^{n×n}$ y un vector $b ∈ R^n$, devuelva el vector $x$ solucion del sistema $Dx = b$.
Alternativamente, el programa puede recibir el vector $d ∈ R^n$ con los valores de la diagonal de $D$, en lugar de la matriz $D$, lo que resulta en general mas eficiente.

In [136]:
def sistemaDiagonal(D,b):
    n=np.shape(D)[0]
    if(D.ndim > 1): #Si la dimension es mayor a 1, significa que D es una matriz
        D=np.diag(D) #Convertimos D en un vector diagonal
    x=np.zeros(n)
    for i in range (n):
        if (D[i]==0):
            if (b[i]!=0):
                return("El sistema no tiene solucion") #No existe x tal que, 0*x=b con b distinto de 0
            else: 
                x[i]=0
        else:
            x[i]=b[i]/D[i] 
    return x

Tomamos $D\in R^{3x3}$ diagonal, $d\in R^{3}$ (vector con los valores de la diagonal de D), y $b\in R^{3}$, vamos a verificar que para D y d el resultado es el mismo.

In [137]:
D=np.array([[1,0,0],[0,2,0],[0,0,3]])
b=np.array([3,9,4])
sistemaDiagonal(D,b)

array([3.        , 4.5       , 1.33333333])

In [138]:
d=np.array([1,2,3])
b=np.array([3,9,4])
sistemaDiagonal(d,b)

array([3.        , 4.5       , 1.33333333])

### Ejercicio 8.
Probar la implementacion realizada de la descomposicion LDLT para resolver un sistema $Qx = b$ tomando una matriz $A ∈ R^{10×10}$ de numeros aleatorios, $Q=A^TA$ y un vector $b ∈ R^{10}$ de numeros aleatorios. Calcular la norma-1 del error $e = Qx̃ -b$ para $x̃$ la solucion calculada.

Para resolver el sistema $LDL^Tx=b$, primero llamaremos $L^Tx=y$, luego tenemos que resolver el sistema $LDy=b$, llamamos $D*y=z$ y nos queda $L*z=b$, asi:
    1) $Lz=b$ 
    2) $Dy=z$
    3) $L^Tx=y$

In [139]:
def sistemaLDLt(A,b):
    
    #Asignamos Q, L, D y L^T
    Q=A.T@A
    L,D = descomposicionLDLt(Q)
    Lt= descomposicionLDLt(Q)[0].T
    
    #L*z=b
    z=sistemaL(L,b)
    
    #D*y=z
    y=sistemaDiagonal(D,z)
    
    #L^T*x=y
    x=sistemaU(Lt, y)
    
    #Calculamos el error
    error=Q@x-b
    
    return [Q,x,b,np.linalg.norm(error,ord=1)]

Tomamos $A\in R^{10x10}$ y $b\in R^{10}$, con valores aleatorios entre 0 y 1, y calculamos el error

In [140]:
A=np.random.rand(10,10)
b=np.random.rand(10)

print("La norma 1 del error es: ", sistemaLDLt(A,b)[3])

La norma 1 del error es:  4.551914400963142e-14


### Ejercicio 9.
Calcular la descomposicion $LU$ de la matriz $Q$. ¿Que relacion encuentra entre ambas descomposiciones?.
En los siguientes ejercicios, queremos verificar que en general la resolucion de sistemas dados por matrices definidas positivas son mas estables que los sistemas generales.

In [141]:
#Tomamos como A y b, la matriz y el vector aleatorio generado anteriormente
Q=sistemaLDLt(A,b)[0]
L,U=descomposicionLU(Q)

Ld, D= descomposicionLDLt(Q)
Lt=descomposicionLDLt(Q)[0].T

Después de comparar L,U,Ld,D,Lt vemos que  $U=D*L^T$, con la diagonal de U y D iguales, y $L^T$ con unos en la diagonal, y mismos elementos que U por encima de la diagonal, verifiquemos esto en python:

In [142]:
print(np.all(np.isclose(np.diag(D),np.diag(U))))
print(np.all((np.isclose(U,D@L.T))))

True
True


### Ejercicio 10.
A partir de la matriz $Q$ construida en el ejercicio 8, construir una matriz $W$ intercambiando las columnas de $Q$ colocando las primeras 5 columnas de Q al final. Es decir $W_i = Q_{i+5}$ para 1 ≤ i ≤ 5 y $W_i = Q_{i−5}$ para 6 ≤ i ≤ 10 (donde Q_i y W_i son las columnas de Q y W respectivamente).
Calcular la descomposicion $LU$ de $W$ y utilizarla para resolver el sistema $Wx = b$. Calcular la norma-1 del error $e = Wx̃− b$ para x̃ la solucion calculada.
¿En cual de los dos casos obtuvo menor norma del error?


In [143]:
def modulo(n):
    if (n<0):
        return -n
    return n

In [144]:
#Ya que, en este ej y en el 12, debemos intercambiar las columnas, creamos esta funcion que nos sirve para ambos.
def intercambiarColumnasN(Q):
    n=np.shape(Q)[0]
    W=np.zeros([n,n])
    for i in range (n):
        W[i,:(n//2)]=Q[i,(n//2):]
        W[i,(n//2):]=Q[i,:(n//2)]
    return W

In [145]:
def sistemaW(A,b):
    
    Q=sistemaLDLt(A,b)[0]
    
    #Intercambiamos las filas de Q
    W=intercambiarColumnasN(Q)
    
    b=sistemaLDLt(A,b)[2]
    
    #W*x=b
    x=sistemaUsandoLU(W,b)
    
    error=W@x-b
    
    print("La norma 1 del error de W*x=b es: ", np.linalg.norm(error,ord=1))
    
    #Veamos cual obtiene menor norma:
    if (np.linalg.norm(error,ord=1)>sistemaLDLt(A,b)[3]):
        print("El sistema LDLt tiene menor error")
    else:
        print("El sistema LU tiene menor error")
    
    diferenciaDeErrores=np.linalg.norm(error,ord=1)-sistemaLDLt(A,b)[3]
    
    print("La diferencia de los errores es: ", modulo(diferenciaDeErrores))

In [146]:
A=np.random.rand(10,10)
b=np.random.rand(10)
sistemaW(A,b)

La norma 1 del error de W*x=b es:  4.2099657093785936e-13
El sistema LDLt tiene menor error
La diferencia de los errores es:  2.220446049250313e-15


Vemos que el error menor va variando, aunque la diferencia entre los errores no es significativa.

### Ejercicio 11.
Calcular mediante comandos de Python la condicion-1 de las matrices $Q$ y $W$. ¿Que relacion observa? ¿Como puede justificarlo?
Para obtener una conclusion mas confiable de los experimentos, repetimos el experimento 100 veces en matrices de mayor tamaño y calculamos la norma promedio de los errores.

In [147]:
def condicionDeMatrices(A,b):
    
    Q=sistemaLDLt(A,b)[0]
    W=intercambiarColumnasN(Q)
    
    cond1Q=np.linalg.cond(Q,1)
    cond1W=np.linalg.cond(W,1)
    
    return cond1Q,cond1W

In [148]:
A=np.random.rand(10,10)
b=np.random.rand(10)
condicionDeMatrices(A,b)

(190381475270.22794, 190381585021.13458)

Como $cond_1(Q)$=$||Q||_1$*$||Q^-1||_1$, y sabemos que la condicion 1 es, en otras palabras, la columna que tiene la suma de modulos mayor, y como $W$ es una permutacion de columnas de $Q$, $||W||_1=||Q||_1$, y en la inversa ocurre algo similar, aunque no son iguales, son muy parecidas.

In [149]:
def experimento(n):
    
    for i in range(100):
        A=np.random.rand(n,n)
        Q=A.T@A
        W=intercambiarColumnasN(Q)

        sumaNormaPromedio=0

        cond1Q=np.linalg.cond(Q,1)
        cond1W=np.linalg.cond(W,1)

#         for i in range (100):
        error=modulo(cond1Q - cond1W)
        sumaNormaPromedio=sumaNormaPromedio + error

    promedio=sumaNormaPromedio/100
    
    return promedio
        


In [150]:
experimento(100)

2.4646706879138947e-06

En la funcion de experimento que nosotros hicimos solo se pueden ingresar numeros pares, igual nos alcanza para ver que mientras mayor sea el tamaño de $Q$, mayor va a ser el error, osea la diferencia entre $cond_1(Q)$ y $cond_1(W)$

### Ejercicio 12.

In [151]:
def Ejercicio12(n):
    
    sumaQ=0
    sumaW=0
    
    for i in range (100):
        
        A=np.random.rand(100,100)
        b=np.random.rand(100)
    
        Q=A.T@A
    
        x_sistemaLDLt=sistemaLDLt(A,b)[1] #Calculamos sistemaLDLt de A ya que sistemaLDLt nos crea Q=A^T*A
    
        error_LDLt= sistemaLDLt(A,b)[3]
        
        sumaQ=sumaQ + error_LDLt
    
        W=intercambiarColumnasN(Q)
    
        x_sistemaLU=sistemaUsandoLU(W,b)
    
        error_LU=error(W,b)[1]
        
        sumaW=sumaW + error_LU
        
    promedioQ=sumaQ/100
    
    promedioW=sumaW/100
    
    return promedioQ, promedioW

In [152]:
Ejercicio12(0)

(5.709459417357898e-08, 5.319426705280294e-06)

Vemos que nuestro sistema $LDL^T$ tiene menor error, lo cual es lo esperado ya que el algoritmo que utilizamos para la descomposición $LDL^T$ tiene menor inestabilidad numérica que el algoritmo que usamos para la descomposición $LU$. Esto sucede debido a que para hallar la descomposición $LU$ triangulamos la matriz, lo que genera una acumulación de errores mayor a la de simplemente reemplazar valores en la formula $LDL^T$.